In [1]:
%reload_ext autoreload
%autoreload 2

import os
from pathlib import Path
import multiprocessing

import jax
import jax.numpy as jnp
import pandas as pd
import numpy as np
import numpyro
import numpyro.distributions as dist

from hbmep.config import Config
from hbmep.model import Baseline
from hbmep.model.utils import Site as site
from hbmep.utils.constants import RECTIFIED_LOGISTIC

PLATFORM = "cpu"
jax.config.update("jax_platforms", PLATFORM)
numpyro.set_platform(PLATFORM)

cpu_count = multiprocessing.cpu_count() - 2
numpyro.set_host_device_count(cpu_count)
numpyro.enable_x64()


#### Load config

In [2]:
root_path = Path(os.getcwd()).parent.parent.parent.absolute()
toml_path = os.path.join(root_path, "configs/gamma_fix.toml")

config = Config(toml_path=toml_path)

config.BUILD_DIR = "/home/vishu/repos/hbmep-paper/reports/fix"


2023-07-21 13:53:40,827 - hbmep.config - INFO - Verifying configuration ...
2023-07-21 13:53:40,828 - hbmep.config - INFO - Success!


In [3]:
class RectifiedLogistic(Baseline):
    LINK = RECTIFIED_LOGISTIC

    def __init__(self, config: Config):
        super(RectifiedLogistic, self).__init__(config=config)

    def _model(self, subject, features, intensity, response_obs=None):
        intensity = intensity.reshape(-1, 1)
        intensity = np.tile(intensity, (1, self.n_response))

        feature0 = features[0].reshape(-1,)

        n_data = intensity.shape[0]
        n_subject = np.unique(subject).shape[0]
        n_feature0 = np.unique(feature0).shape[0]

        with numpyro.plate(site.n_response, self.n_response, dim=-1):
            with numpyro.plate(site.n_subject, n_subject, dim=-2):
                """ Hyper-priors """
                mu_a = numpyro.sample(
                    site.mu_a,
                    dist.TruncatedNormal(150, 50, low=0)
                )
                sigma_a = numpyro.sample(site.sigma_a, dist.HalfNormal(50))

                sigma_b = numpyro.sample(site.sigma_b, dist.HalfNormal(0.1))

                sigma_L = numpyro.sample(site.sigma_L, dist.HalfNormal(0.05))
                sigma_H = numpyro.sample(site.sigma_H, dist.HalfNormal(5))
                sigma_v = numpyro.sample(site.sigma_v, dist.HalfNormal(10))

                with numpyro.plate("n_feature0", n_feature0, dim=-3):
                    """ Priors """
                    a = numpyro.sample(
                        site.a,
                        dist.TruncatedNormal(mu_a, sigma_a, low=0)
                    )
                    b = numpyro.sample(site.b, dist.HalfNormal(sigma_b))

                    L = numpyro.sample(site.L, dist.HalfNormal(sigma_L))
                    H = numpyro.sample(site.H, dist.HalfNormal(sigma_H))
                    v = numpyro.sample(site.v, dist.HalfNormal(sigma_v))

                    g_1 = numpyro.sample("g_1", dist.Exponential(1 / 100))
                    g_2 = numpyro.sample("g_2", dist.Exponential(1 / 100))

        """ Model """
        mu = numpyro.deterministic(
            site.mu,
            L[feature0, subject]
            + jnp.maximum(
                0,
                -1
                + (H[feature0, subject] + 1)
                / jnp.power(
                    1
                    + (jnp.power(1 + H[feature0, subject], v[feature0, subject]) - 1)
                    * jnp.exp(-b[feature0, subject] * (intensity - a[feature0, subject])),
                    1 / v[feature0, subject]
                )
            )
        )
        beta = numpyro.deterministic(
            site.beta,
            g_1[feature0, subject] + g_2[feature0, subject] * (1 / mu)
        )

        with numpyro.plate(site.data, n_data):
            return numpyro.sample(
                site.obs,
                dist.Gamma(concentration=mu * beta, rate=beta).to_event(1),
                obs=response_obs
            )


model = RectifiedLogistic(config=config)

In [4]:
df = pd.read_csv(model.csv_path)

sub = ["amap01", "amap02", "amap03", "amap04"]
sub += ["amap05", "amap06", "amap07", "amap08"]

ind = df.participant.isin(sub)

df = df[ind].copy()
df.reset_index(drop=True, inplace=True)
print(df.shape)

df, encoder_dict = model.load(df=df)


2023-07-21 13:53:41,257 - hbmep.dataset.core - INFO - Artefacts will be stored here - /home/vishu/repos/hbmep-paper/reports/fix
2023-07-21 13:53:41,257 - hbmep.dataset.core - INFO - Copied config to /home/vishu/repos/hbmep-paper/reports/fix
2023-07-21 13:53:41,258 - hbmep.dataset.core - INFO - Processing data ...
2023-07-21 13:53:41,259 - hbmep.utils.utils - INFO - func:load took: 0.00 sec


(100, 43)


In [5]:
mcmc, posterior_samples = model.run_inference(df=df)


2023-07-21 13:53:41,362 - hbmep.model.baseline - INFO - Running inference with rectified_logistic ...


  0%|          | 0/10000 [00:00<?, ?it/s]

  0%|          | 0/10000 [00:00<?, ?it/s]

  0%|          | 0/10000 [00:00<?, ?it/s]

  0%|          | 0/10000 [00:00<?, ?it/s]

2023-07-21 13:54:20,418 - hbmep.utils.utils - INFO - func:run_inference took: 39.06 sec


In [6]:
mcmc.print_summary(prob=.95)



                mean       std    median      2.5%     97.5%     n_eff     r_hat
  H[0,0,0]      6.01      0.10      6.01      5.82      6.20  17501.71      1.00
  H[0,0,1]      3.78      0.06      3.78      3.67      3.89  23813.35      1.00
  H[1,0,0]      4.28      0.19      4.27      3.92      4.66  12821.53      1.00
  H[1,0,1]      1.49      0.03      1.49      1.42      1.56  21866.55      1.00
  L[0,0,0]      0.05      0.01      0.05      0.03      0.06  17920.35      1.00
  L[0,0,1]      0.05      0.01      0.05      0.04      0.07  23048.75      1.00
  L[1,0,0]      0.05      0.01      0.05      0.04      0.07   6192.25      1.00
  L[1,0,1]      0.02      0.00      0.02      0.02      0.03  20246.87      1.00
  a[0,0,0]    118.72      2.90    118.22    114.21    125.14   7055.26      1.00
  a[0,0,1]    132.20      1.71    132.30    128.65    135.34  20909.66      1.00
  a[1,0,0]    159.15      4.84    159.24    151.52    166.84   1019.90      1.00
  a[1,0,1]    164.74      0

In [7]:
model.render_recruitment_curves(df=df, encoder_dict=encoder_dict, posterior_samples=posterior_samples)


2023-07-21 13:54:20,608 - hbmep.model.baseline - INFO - Rendering recruitment curves ...
2023-07-21 13:54:25,988 - hbmep.model.baseline - INFO - Saved to /home/vishu/repos/hbmep-paper/reports/fix/recruitment_curves.pdf
2023-07-21 13:54:25,988 - hbmep.utils.utils - INFO - func:render_recruitment_curves took: 5.38 sec


In [8]:
model.render_predictive_check(df=df, encoder_dict=encoder_dict, posterior_samples=posterior_samples)


2023-07-21 13:54:26,050 - hbmep.model.baseline - INFO - Rendering Posterior Predictive Check ...


2023-07-21 13:54:32,329 - hbmep.model.baseline - INFO - Saved to /home/vishu/repos/hbmep-paper/reports/fix/posterior_predictive_check.pdf
2023-07-21 13:54:32,332 - hbmep.utils.utils - INFO - func:render_predictive_check took: 6.28 sec
